In [ ]:
!conda install gdcm -c conda-forge -y

In [ ]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut


def read_xray(path,voi_lut=True,fix_monochrome=True):
    dicom=pydicom.read_file(path)
    
    if voi_lut:
        data=apply_voi_lut(dicom.pixel_array,dicom)
    
    else:
        data=dicom.pixel_array
        
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    
    data=data-np.min(data)
    data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
train=pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')

In [ ]:
path = '../input/siim-covid19-detection/train/ae3e63d94c13/288554eb6182/e00f9fe0cce5.dcm'
dicom = pydicom.read_file(path)

In [ ]:
'''
os.walk()
return 3 tuples: root,dirs,files
'''

In [ ]:
image_id=[]
dimension0=[];dimension1=[]
splits=[]


for split in ['train','test']:
    
    save_dir=f'/kaggle/tmp/{split}/'
    
    os.makedirs(save_dir,exist_ok=True)

    for dirname,_,filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            xray=read_xray(os.path.join(dirname,file))
            im=resize(xray,size=512) ## resize to 512
            im.save(os.path.join(save_dir,file.replace('dcm','jpg')))
            
            image_id.append(file.replace('.dcm',''))
            dimension0.append(xray.shape[0])
            dimension1.append(xray.shape[1])
            splits.append(split)

In [ ]:
%%time
!tar -zcf train.tar.gz -C "/kaggle/tmp/train/" .
!tar -zcf test.tar.gz -C "/kaggle/tmp/test/" .

In [ ]:
df = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dimension0, 'dim1': dimension1, 'split': splits})
df.to_csv('meta.csv', index=False)